# Compare Data Sources on Buildings

Pull data from different sources for a small area. Save results in CSV and GeoJSON formats. 

Do not use the flask_app/myenv virtual environment to run this notebook. Create a different virtual environment using the `requirements-notebooks.txt` file. 

This notebook also relies on USACE and FEMA data downloaded to different parts of your local filesystem. I haven't automated that part, but I might in the future. 

In [ ]:
# Main libraries
import folium
import pandas as pd
import geopandas as gpd
from shapely import box
from pathlib import Path

# Microsoft
import mercantile
import sys

sys.path.append("../../flask_app")

from flask_app.utils.update_dataset_links import update_dataset_links
from flask_app.utils.update_quadkeys import update_quadkeys

# OSM
import osmnx as ox

In [ ]:
# Confirm that the Utils path is from this repo. This path should end in: '/cbl-web-tool/flask_app/utils/__init__.py'

# import flask_app.utils
# Path(utils.__file__).resolve()

In [ ]:
LOCAL_CRS = "EPSG:3502"  # NAD83(NSRS2007) / Colorado Central (ftUS)
WEB_CRS = "EPSG:4326"

In [ ]:
# lat, lon
# samples_min = [bottom, left]
# samples_max = [top, right]

samples_dir = Path("samples_golden")
sample_county = "Jefferson"
sample_min = [39.731257573115556, -105.24554574681486]  # Lookout Mountain
sample_max = [39.768608073891706, -105.1641782613617]  # Applewood

# samples_dir = Path('samples_denver_downtown')
# sample_county = 'Denver'
# sample_min = [39.740289, -105.015891] # Colfax and Zuni, approx
# sample_max = [39.76921498291726, -104.98196890096911] # Delgany and Festival St

# samples_dir = Path('samples_denver_athmar_park')
# sample_county = 'Denver'
# sample_min = [39.6967070971316, -105.02507698812224] # Federal and Mississippi
# sample_max = [39.70392950655037, -105.0086309590224] # Raritan and Exposition
# there's a weird north-south strip of missing data from the MS Footprints here.

samples_dir.mkdir(exist_ok=True)
sample_box = box(
    xmin=sample_min[1], ymin=sample_min[0], xmax=sample_max[1], ymax=sample_max[0]
)

sample_gdf_web = gpd.GeoDataFrame(index=[0], crs=WEB_CRS, geometry=[sample_box])
sample_gdf_local = sample_gdf_web.to_crs(LOCAL_CRS)
sample_gdf_local.to_file(samples_dir / "sample_box.geojson", driver="GeoJSON")

In [ ]:
comparison = pd.read_csv("Building_Data_Sources.csv")

In [ ]:
this_map = folium.Map(prefer_canvas=True)
folium.GeoJson(sample_box).add_to(this_map)
this_map.fit_bounds(this_map.get_bounds())
# this_map

## 1. Microsoft - Global Building Footprints 

https://github.com/microsoft/GlobalMLBuildingFootprints 

Most of this code is adapted from `Microsoft Footprints Savannah.ipynb`

Sometimes there is missing data in MS Footprints in a north-south strip. Not sure why! Keep an eye out for it.

In [ ]:
# Find all quadkeys that the coordinates fall within
quadkeys = set()
for coords in [sample_min, sample_max]:
    tile = mercantile.tile(coords[1], coords[0], 9)
    quadkey = int(mercantile.quadkey(tile))
    quadkeys.add(quadkey)

quadkeys

In [ ]:
# Download quadkey dataset links
update_dataset_links()

# Download quadkeys
update_quadkeys(list(quadkeys))

In [ ]:
import gzip
from typing import Any

# Load building shapes for each quadkey into memory
loaded_quadkeys: dict[int, Any] = {}

for q in quadkeys:
    if q not in loaded_quadkeys:
        print(f"Loading quadkey: {q}")

        with gzip.open(f"data/quadkeys/{q}.geojsonl.gz", "rb") as f:
            loaded_quadkeys[q] = gpd.read_file(f)
            print(f"  {len(loaded_quadkeys[q])} Microsoft footprints in quadkey")

In [ ]:
matching_ms_local_dict = {}

print("Matching parcel shapes to Microsoft footprints.")

for q in quadkeys:
    print(f"Quadkey: {q}")
    quadkey_local = loaded_quadkeys[q].to_crs(LOCAL_CRS)

    nearest = gpd.sjoin_nearest(
        sample_gdf_local, quadkey_local, distance_col="point_to_building_feet"
    )
    nearest.rename(columns={"index_right": "index_ms"}, inplace=True)

    matching_ms_local_dict[quadkey] = quadkey_local[
        quadkey_local.index.isin(nearest.index_ms)
    ]
    matching_ms_local_dict[quadkey] = (
        matching_ms_local_dict[quadkey]
        .reset_index()
        .rename(columns={"index": "index_ms"})
    )

In [ ]:
matching_ms_local = (
    pd.concat(matching_ms_local_dict)
    .reset_index(names=["ms_quadkey", "index_to_drop"])
    .drop(["index_to_drop"], axis=1)
)

In [ ]:
# this_map = folium.Map(prefer_canvas=True)
# folium.GeoJson(sample_box).add_to(this_map)
# folium.GeoJson(matching_ms_local.to_crs(WEB_CRS)).add_to(this_map)
# this_map.fit_bounds(this_map.get_bounds())
# # this_map

In [ ]:
matching_ms_local.to_csv(samples_dir / "ms_footprints.csv", index=False)
matching_ms_local.to_file(samples_dir / "ms_footprints.geojson", driver="GeoJSON")

In [ ]:
comparison.loc[
    comparison.Name == "Global ML Building Footprints", "sample_buildings"
] = len(matching_ms_local)
comparison.loc[
    comparison.Name == "Global ML Building Footprints", "sample_footprint_area"
] = matching_ms_local.geometry.area.sum()

## 2. Open Street Maps

In [ ]:
# sample = bottom, left, top, right
# osm wants = Bounding box as (left, bottom, right, top)

matching_osm = ox.features_from_bbox(
    bbox=(sample_min[1], sample_min[0], sample_max[1], sample_max[0]),
    tags={"building": True},
)

In [ ]:
# Define a function to calculate the new field based on index values
# You might need to be off of the VPN / office network to make this connection
def calculate_osm_url_field(row):
    return f"https://www.openstreetmap.org/{row.name[0]}/{row.name[1]}"


# Apply the function to each row to calculate the new field
matching_osm["osm_url"] = matching_osm.apply(calculate_osm_url_field, axis=1)

# convert the height to None or float
if "height" in matching_osm.columns:
    matching_osm["height"] = pd.to_numeric(matching_osm["height"], errors="coerce")

# reset the index
matching_osm.reset_index(inplace=True)

# Convert to local for area calculation
matching_osm_local = matching_osm.to_crs(LOCAL_CRS)

In [ ]:
matching_osm.to_csv(samples_dir / "osm.csv", index=False)
# this could be split into two files: one for points (few) and one for polygons (most)
matching_osm.to_file(samples_dir / "osm.geojson", driver="GeoJSON")

In [ ]:
comparison.loc[comparison.Name == "OpenStreetMap", "sample_buildings"] = len(
    matching_osm
)
comparison.loc[comparison.Name == "OpenStreetMap", "sample_footprint_area"] = (
    matching_osm_local.geometry.area.sum()
)

## 3. ORNL - LandScan

https://landscan.ornl.gov/

In [ ]:
# skipping

## 4. USACE - National Structure Inventory

https://www.hec.usace.army.mil/confluence/nsi

In [ ]:
if sample_county == "Denver":
    nsi = gpd.read_file(
        "~/OneDrive - NREL/DevinDocuments/National_Structure_Inventory_USACE/fips_08031.geojson"
    )
elif sample_county == "Jefferson":
    nsi = gpd.read_file(
        "~/OneDrive - NREL/DevinDocuments/National_Structure_Inventory_USACE/fips_08059.geojson"
    )
else:
    error_str = f"County {sample_county} not downloaded."
    raise Exception(error_str)

In [ ]:
nsi_local = nsi.to_crs(LOCAL_CRS)

In [ ]:
nearest_nsi = gpd.sjoin_nearest(
    sample_gdf_local, nsi_local, distance_col="point_to_building_feet"
)

matching_nsi = nsi_local[nsi_local.index.isin(nearest_nsi.index_right)]

In [ ]:
# this_map = folium.Map(prefer_canvas=True)
# folium.GeoJson(sample_box).add_to(this_map)
# folium.GeoJson(matching_nsi).add_to(this_map)
# this_map.fit_bounds(this_map.get_bounds())
# # this_map

In [ ]:
matching_nsi.to_csv(samples_dir / "usace_nsi.csv", index=False)
matching_nsi.to_file(samples_dir / "usace_nsi.geojson", driver="GeoJSON")

In [ ]:
comparison.loc[
    comparison.Name == "National Structure Inventory", "sample_buildings"
] = len(matching_nsi)
comparison.loc[
    comparison.Name == "National Structure Inventory", "sample_footprint_area"
] = matching_nsi.sqft.sum()

## 5. FEMA USA Structures

https://gis-fema.hub.arcgis.com/pages/usa-structures 

In [ ]:
fema_dir = Path("~/OneDrive - NREL/DevinDocuments/FEMA Buildings/")
fema = gpd.read_file(fema_dir / "data/Deliverable20230630CO/CO_Structures.gdb")

# fema_denver = fema[fema.FIPS == '08031'].copy()
# fema_denver.to_file(fema_dir / 'fema_denver.geojson', driver='GeoJSON')
# fema = gpd.read_file(fema_dir / 'fema_denver.geojson')

In [ ]:
fema_local = fema.to_crs(LOCAL_CRS)

In [ ]:
nearest_fema = gpd.sjoin_nearest(
    sample_gdf_local, fema_local, distance_col="point_to_building_feet"
)

matching_fema = fema_local[fema_local.index.isin(nearest_fema.index_right)]

In [ ]:
# this_map = folium.Map(prefer_canvas=True)
# folium.GeoJson(sample_box).add_to(this_map)
# folium.GeoJson(matching_fema['geometry']).add_to(this_map)
# this_map.fit_bounds(this_map.get_bounds())
# # this_map

In [ ]:
fema.OCC_CLS.value_counts().head()

In [ ]:
fema.PRIM_OCC.value_counts().head()

In [ ]:
pivot = pd.pivot_table(
    data=fema,
    index=["PRIM_OCC"],
    columns=["OCC_CLS"],
    margins=True,
    values=["BUILD_ID"],
    aggfunc="count",
    fill_value=0,
)

# pivot.to_clipboard()

In [ ]:
prim_occ_cls_counts = fema.groupby(["OCC_CLS", "PRIM_OCC"]).size()
# prim_occ_cls_counts.to_clipboard()

In [ ]:
prim_occ_cls_sqft = fema.groupby(["OCC_CLS", "PRIM_OCC"]).SQFEET.sum()
# prim_occ_cls_sqft.to_clipboard()

In [ ]:
prim_occ_cls = fema.groupby(["OCC_CLS", "PRIM_OCC"]).agg(
    num_buildings=("BUILD_ID", "count"), sqft_sum=("SQFEET", "sum")
)
prim_occ_cls.to_clipboard()

In [ ]:
matching_fema.to_csv(samples_dir / "fema_usa_structures.csv", index=False)
matching_fema.to_file(samples_dir / "fema_usa_structures.geojson", driver="GeoJSON")

In [ ]:
comparison.loc[comparison.Name == "USA Structures", "sample_buildings"] = len(
    matching_fema
)
comparison.loc[comparison.Name == "USA Structures", "sample_footprint_area"] = (
    matching_fema.SQFEET.sum()
)

## Results

In [ ]:
pd.options.display.float_format = "{:,.0f}".format
comparison[["Name", "Owner", "sample_buildings", "sample_footprint_area"]]

In [ ]:
comparison.to_csv(samples_dir / "comparison_counts.csv", index=False)